In [3]:
from ROOT import TCanvas, TFile, TProfile, TNtuple, TH1F, TH2F,TChain,TGraph,TDatime,TLine,TLatex,TPad,TLegend,TPaveText,TF1,TGraphErrors,TGaxis
from ROOT import gROOT, gBenchmark, gRandom, gSystem, gPad,gStyle
from io import StringIO
import numpy as np
import ctypes
import os
from array import *
import math
import sys
from prettytable import PrettyTable
from decimal import Decimal, getcontext
import mpmath
import sympy as sp
import ezodf
from barion.amedata import *
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from adjustText import adjust_text


In [ ]:
def gaussian(x, amplitude, mean, stddev):
    return amplitude * np.exp(-((x - mean) / stddev)**2 / 2)

In [ ]:
def Read_AME(file_path = "AME.rd"):
    try:
        # Try to open the file and read its contents
        with open(file_path, 'r') as file:
            lines = file.readlines()
    except FileNotFoundError:
        # If the file is not found, print an error message
        print(f"Error: File '{file_path}' not found.")
        return
    print(f"Reading mass data from file '{file_path}'...")
    # Initialize an empty list to store parsed data
    # Iterate through each line and parse the data
    data_list= []
    for line in lines:
        line = line.strip()  # Remove leading and trailing whitespace
        if line:  # Skip empty lines
            tokens = line.split()  # Split the line by spaces
            if len(tokens) == 6:  # Ensure each line has 6 fields
                entry = {
                    'Element': tokens[0],
                    'Z': int(tokens[1]),
                    'A': int(tokens[2]),
                    'ME (keV)': float(tokens[3]),
                    'ME_error (keV)': float(tokens[4]),
                    'Symbol': tokens[5] #this is not necessary
                }
                data_list.append(entry)
    return data_list

In [ ]:
def Read_AME_barion():
    ame = AMEData()  
    data_list= []
    for i in ame.ame_table:
        entry = {
            'Element': str(i[5])+i[6],
            'Z': int(i[4]),
            'A': int(i[5]),
            'ME (keV)': float(i[8]),
            'ME_error (keV)': float(i[9])
                }
        data_list.append(entry)
    return data_list

In [ ]:
def Read_elbien_file(file_path):
    data_list = []
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return data_list
    print(f"Reading electron binding energy data from file '{file_path}'...")
    header_line = None
    for line in lines:
        if line.startswith('#'):
            if header_line is None:
                header_line = line
            continue
        
        if not line.strip():
            continue  # Skip empty lines
        
        tokens = line.split()
        if header_line:
            headers = header_line.strip().split()[1:]  # Extract column headers
            entry = {'Z': int(tokens[0])}
            
            entry['TotBEn'] = int(tokens[1])
            # Add more fields up to '105e-ion'
            for i in range(len(headers), len(tokens)-2):
                entry[f'{i + 1}e-ion'] = int(tokens[i+2])
            data_list.append(entry)
    return data_list

In [ ]:
def Read_revlutiontime(file_path = "revlutiontime.txt"):
    try:
        # Try to open the file and read its contents
        with open(file_path, 'r') as file:
            lines = file.readlines()
    except FileNotFoundError:
        # If the file is not found, print an error message
        print(f"Error: File '{file_path}' not found.")
        return
    print(f"Reading revolution time from file '{file_path}'...")
    # Initialize an empty list to store parsed data
    # Iterate through each line and parse the data
    data_list = []
    for line in lines:
        line = line.strip()  # Remove leading and trailing whitespace
        if line.startswith('#'):
            continue
        if line:  # Skip empty lines
            tokens = line.split()  # Split the line by spaces
            entry = {
                'Element': tokens[0],
                'Z': int(float(tokens[1])),
                'Q': int(float(tokens[2])),
                'Flag': tokens[3],
                'RevolutionTime[ns]': float(tokens[4]),
                'Count': float(tokens[5]),
                'width of revolutionTime[ps]': float(tokens[6]),
                'error of width of revolutionTime[ps]': float(tokens[7])
            }
            data_list.append(entry)
    
    # Print the parsed data
    print("Total nuclei number: ",len(data_list))
    for entry in range (0, len(data_list)):
        print(data_list[entry])
    print("...")
        
    return data_list

In [ ]:
def GetAMEData(AME_data,element):    
    ME     =0
    MEError=0
    A      =0
    N      =0
    for entry_AME in AME_data:
        element_AME= entry_AME['Element']
        Z_AME      = entry_AME['Z']
        A_AME      = entry_AME['A']
        ME_AME     = entry_AME['ME (keV)']/1e3 # MeV
        MEError_AME= entry_AME['ME_error (keV)']/1e3 # MeV
        
        if element == element_AME and Z == Z_AME:
            ME      = ME_AME
            MEError = MEError_AME
            A       = A_AME
            N       =  A_AME - Z_AME
            
    if A==0:
        print(f"Error! The Z={Z} of {element} is incorrect. Please check it.")
    return ME, MEError, A, N

In [ ]:
def GetBindingEnergy(elbien_data,Z,Q):
    EBindingEnergy = 0
    for entry_elbien in elbien_data:
        Z_elbien      = entry_elbien['Z']
        if  Z == Z_elbien:
            a = ''
            if Z-Q == 0:
                a = 'TotBEn'
            else:
                a = str(Z-Q) + 'e-ion'
            EBindingEnergy = entry_elbien[a]/1e6
    return EBindingEnergy

In [ ]:
def LeastSquareFit(T, TError, MoQ, MoQError, p,iterationMax,A0,OutputMatrixOrNot,tol = None):
    N        =len(T)
    A        =A0[:]
    chi2_min =1e20
    A_min    =A[:]
    b2_inverse_min =[]
    chi2_previous = 0
    for iteration in range(0, iterationMax):
        FDB=[]
        PDB=[]
        # step 1: F matrix. This can go outside the loof
        for i in range(0, N):
            f   = T[i]
            row = []
            for k in range(0, p):
                row.insert(k,mpmath.power(f, k))
            FDB.append(row)
        # step 2 delta_MoQ_fi
        delta_MoQ_fiDB=[]
        for i in range(0, N):
            fi           = T[i]
            delta_fi     = TError[i]
            delta_MoQ_fi = 0
            for k in range(1, p):
                delta_MoQ_fi = delta_MoQ_fi + k*A[k]*mpmath.power(fi, k - 1)*delta_fi
            delta_MoQ_fiDB.insert(i,delta_MoQ_fi)
        #step 3 P matrix
        for i in range(0, N):
            row = []
            for j in range(0, N):
                a=0
                if i ==j:
                    a = 1/(float(MoQError[i])**2 + float(delta_MoQ_fiDB[i])**2)
                row.insert(j, a)
            PDB.insert(i,row)
        # step 4 chi2
        if iteration==0:
            chi2 = 0
            for i in range(0, N):
                fi        = T[i]
                delta_fi  = TError[i]
                y         = MoQ[i]
                ye        = MoQError[i]
                yfit      = 0
                yfit_error=delta_MoQ_fiDB[i]
                for k in range(0, p):
                    yfit = yfit + A[k]*fi**(k)
                chi2 = chi2 + (y - yfit)**2 / (ye**2  + yfit_error**2)
            chi2_min=chi2
            A_min   =A[:]
            if OutputMatrixOrNot: print("iteration  = ",iteration," A_min = ",A_min," chi2_min=", chi2_min)
            
        # step 5 calculated parameters A
        F = mpmath.matrix(FDB)
        P = mpmath.matrix(PDB)
        F_T = F.transpose()
        a1 = P *  mpmath.matrix(MoQ)
        a2 = F_T * a1
        b1 = P * F
        b2 = F_T * b1
        b2_sympy = sp.Matrix(b2.tolist())
        det_b2 = b2_sympy.det()
        chi2 = 0        
        if det_b2 == 0:
            print("b2 = FT*P*F is a singular matrix. Iteration stopped.")
            break
        else:
            b2_inverse = b2**-1
            A = b2_inverse * a2
            if iteration == 0: b2_inverse_min  = b2_inverse.copy()
            # step 6
            for i in range(0, N):
                fi        = T[i]
                delta_fi  = TError[i]
                y         = MoQ[i]
                ye        = MoQError[i]
                yfit      = 0
                yfit_error=delta_MoQ_fiDB[i]
                for k in range(0, p):
                    yfit = yfit + A[k]*fi**(k)
                chi2 = chi2 + (y - yfit)**2 / (ye**2  + yfit_error**2)
            if chi2_min>=chi2:
                chi2_min=chi2
                A_min   =A[:]
                b2_inverse_min  = b2_inverse.copy()
                if OutputMatrixOrNot: print("iteration  = ",iteration," A_min = ",A_min," chi2_min=", chi2_min)
                #iteration = iteration
                
        delta_chi2 = abs(chi2 - chi2_previous)
        if tol:
            if delta_chi2 < tol:
                break
        chi2_previous = chi2
        if OutputMatrixOrNot: print_output(iteration, A, p, N, MoQ, MoQError, delta_MoQ_fiDB, T, TError, F, F_T, PDB, a1, a2, b1, b2, b2_inverse, chi2,A_min, chi2_min)
        print("iteration  = ",iteration," A_min = ",A_min," chi2_min=", np.round(float(chi2_min)), "delta_chi2 =",np.round(float(delta_chi2))) 
    return A_min, chi2_min,b2_inverse_min

In [ ]:
def print_output(iteration, A, p, N, MoQ, MoQError, delta_MoQ_fiDB, T, TError, F, F_T, PDB, a1, a2, b1, b2, b2_inverse, chi2, A_min, chi2_min):
    print(f"Iteration {iteration} - Best Fit Parameters:")
    for k in range(p):
        print(f"A[{k}] = {float(A_min[k]):.5e}")

    print("\nData and Uncertainties:")
    print("{:<15} {:<15} {:<15} {:<15}".format("MoQ", "MoQError", "delta_MoQ_fi", "T +/- TError"))
    for i in range(N):
        moq_val = float(MoQ[i])
        moq_err = float(MoQError[i])
        delta_moq_fi = float(delta_MoQ_fiDB[i])
        t_val = float(T[i])
        t_err = float(TError[i])

        print("{:<15.5f} {:<15.5e} {:<15.5e} {:<15.5f} +/- {:<15.5f}".format(moq_val, moq_err, delta_moq_fi, t_val, t_err))

    print("\nMatrices:")
    print("F:")
    for i in range(N):
        print("\t".join(f"{float(F[i, k]):.3f}" for k in range(p)))

    print("F_T:")
    for i in range(p):
        print("\t".join(f"{float(F_T[i, k]):.3e}" for k in range(N)))

    print("P:")
    for i in range(N):
        for j in range(N):
            pdb_val = float(PDB[i][j])
            if i == j:
                print("{:<6.3f}".format(pdb_val), end="\t")
            else:
                print("{:<1}".format(pdb_val), end="\t")
        print()

    print("\nIntermediate Results:")
    print("a1 = P * MoQ:")
    for i in range(N):
        print(f"{float(a1[i]):.5f}")

    print("a2 = F_T * P * MoQ:")
    for i in range(p):
        print(f"{float(a2[i]):.5f}")

    print("b1 = P * F:")
    for i in range(N):
        print("\t".join(f"{float(b1[i, k]):.3f}" for k in range(p)))

    print("b2 = F_T * P * F:")
    for i in range(p):
        print("\t".join(f"{float(b2[i, k]):.3f}" for k in range(p)))

    print("b2_inverse = (F_T * P * F)^-1:")
    for i in range(p):
        print("\t".join(f"{float(b2_inverse[i, k]):.3e}" for k in range(p)))

    print("\nFinal Chi-Squared:")
    print(f"Chi-squared: {float(chi2):.5f} (Minimum: {float(chi2_min):.5f})")


In [ ]:
def MassCalibration(Nuclei_Y,Nuclei_N,OutputMatrixOrNot_Y,p,A0_Y,iterationMax,tol = 0.1):
    Y_Y        = []
    YError_Y   = []
    X_Y        = []
    XError_Y   = []
    for i in range(0, len(Nuclei_Y)):
        Y_Y        .insert(i,Nuclei_Y[i][13])
        YError_Y   .insert(i,Nuclei_Y[i][14])
        X_Y        .insert(i,Nuclei_Y[i][6])
        XError_Y   .insert(i,Nuclei_Y[i][9])
    
    A_min_Y, chi2_min_Y,b2_inverse_min_Y = LeastSquareFit (X_Y, XError_Y, Y_Y, YError_Y, p,iterationMax,A0_Y, OutputMatrixOrNot_Y,tol)
    # Nuclei_N.insert(i_N,[element,Z,A,N,Q, Flag,T, Count, SigmaT, TError, ME, MEError, EBindingEnergy, Mass_Q, Mass_QError])
    
    table_list = []
    for i in range(0, len(Nuclei_N)):
        # it is not f but T
        f        = Nuclei_N[i][6]
        delta_f  = Nuclei_N[i][9]

        MoQ_N    = 0
        for k in range(0, p):
            MoQ_N = MoQ_N +  A_min_Y[k]*(f**k)

        MoQ_N_fit_av=0
        for k in range(0, p):
            for l in range(0,p):
                MoQ_N_fit_av = MoQ_N_fit_av + b2_inverse_min_Y[k,l]*(f**k)*(f**l)
        MoQ_N_fit_av   = MoQ_N_fit_av**0.5

        n = len(Y_Y)
        MoQ_N_fit_sca  = (chi2_min_Y/(n-p))**0.5*MoQ_N_fit_av
        MoQ_N_fit_error= max(MoQ_N_fit_av, MoQ_N_fit_sca) # this formula is given in matos's thesis page 67.
        #MoQ_N_fit_error= MoQ_N_fit_av  # this formular is used the old fortran code.

        MoQ_N_fre_error= 0
        for k in range(1, p):
            MoQ_N_fre_error = MoQ_N_fre_error +  k*A_min_Y[k]*(f**(k-1))*delta_f

        MoQ_N_tot_error = (MoQ_N_fit_error**2 + MoQ_N_fre_error**2)**0.5
        element          = Nuclei_N[i][0]
        Z_N              = Nuclei_N[i][1]
        A_N              = Nuclei_N[i][2]
        N_N              = Nuclei_N[i][3]
        Q_N              = Nuclei_N[i][4]
        Flag             = Nuclei_N[i][5]
        ME_N_AME         = Nuclei_N[i][10] # MeV
        MEError_N_AME    = Nuclei_N[i][11] # MeV
        EBindingEnergy_N = Nuclei_N[i][12] # MeV
        MoQ_N_AME        = Nuclei_N[i][13] # u/e
        MoQError_N_AME   = Nuclei_N[i][14] # u/e
        Mass_N           = MoQ_N * Q_N * amu 
        ME_N             = Mass_N - A_N*amu + Q_N*me - EBindingEnergy_N#EBindingEnergy # MeV *********************
        MEError_N        = MoQ_N_tot_error/MoQ_N * Mass_N # MeV
        table_list.append([element, Flag,Z_N, A_N,Q_N, f/1000, (ME_N*1000-ME_N_AME*1000), (ME_N*1000), (MEError_N*1000), (MEError_N_AME*1000)])
    return table_list,A_min_Y,chi2_min_Y

In [ ]:
def plot_residual_distribution(bx,by,bye, bins = 15):
    plt.style.use('/home/duskdawn/analysis/plt-style.mplstyle')
    plt.rc('axes', unicode_minus=False)
    
    fig, axs = plt.subplots(1, 1, figsize=(25, 12))
    bins = bins
    hist, bin_edges = np.histogram(by, bins=bins, density=True)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    
    initial_params = [110.0, 15.0, 12.0]  # Initial parameter guesses: amplitude, mean, stddev
    fit_params, err = curve_fit(gaussian, bin_centers, hist, p0=initial_params)
    x_bins_fit = np.arange(bin_centers.min()*10000,bin_centers.max()*10000,1000)/10000
    
    xerr_bars = []
    for i in range(len(bin_centers)):
        bin_mask = (by >= bin_edges[i]) & (by < bin_edges[i + 1])
        bin_y_error = bye[bin_mask]
        if len(bin_y_error) > 0:  # Check if the bin is not empty
            x_error_avg = np.mean(bin_y_error)
            xerr_bars.append(x_error_avg)
        else:
            xerr_bars.append(0.0)  # Assign a default value for empty bins
        
    width = (bin_edges[1] - bin_edges[0]) -0.2
    plt.bar(bin_centers,  hist/ hist.max(), width=width, color='r', xerr=xerr_bars,
           edgecolor='red', linewidth=2 )
    plt.errorbar(bin_centers,hist / hist.max(),xerr=xerr_bars,fmt='bo',ecolor='black',elinewidth=5,capsize=4,alpha=0.3,markersize='13')
    plt.plot(x_bins_fit, gaussian(x_bins_fit, *fit_params)/ hist.max(), 
             'b-', label=f'mean = {np.round(fit_params[1])}   std = {np.round(fit_params[2])}', linewidth = 3)
    plt.title('Residuals distribution',  fontsize=38)
    plt.xlabel(r"$\Delta_{exp} - \Delta_{AME}$ (keV)", fontsize=34)
    plt.ylabel('Normalized amplitud', fontsize=34)
    plt.xticks(fontsize=34)
    plt.yticks(fontsize=34)
    plt.legend()
    plt.show()

In [ ]:
def convert_name(name):
    import re
    # Use regex to extract the atomic number, isotope name, and charge
    atomic_num, element, charge = re.findall(r"\d+|\D+", name)
    # Format the atomic number and charge as superscripts
    atomic_num = '$^{'+str(atomic_num)+'}$'
    charge = '$^{'+str(charge)+'+}$'

    # Form the element symbol by capitalizing the first letter of the isotope name
    element = element.capitalize()

    # Combine the superscripts and element symbol
    return atomic_num + element + charge

In [ ]:
def define_precision(p):
    if p ==4:
        precision = 31
    elif p==5:
        precision = 42
    elif p ==3:
        precision = 18
    elif p == 6:
        precision = 54
    elif p == 7:
        precision = 65
    else:
        precision = None
    return precision

In [ ]:
print("Welcome to the mass calibration code :) ")
print("I hope you get some exciting measurements! ")